In [2]:
import os
import requests
from openai import OpenAI

# ✅ OpenAI API 키 직접 지정
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ✅ 캠핑장 공공 API 키
CAMP_API_KEY = "DCvakj3KUyfmU0c%2FF7CbsLX4VXkEjGdcDfj1A0tFvZzJZL9h70OoFvmEeWVg54OdoXH7mOXwxIHM45Mx6IVxlA%3D%3D"  # 공공데이터포털에서 받은 키 (URL 인코딩 필요)

# ✅ 1. 캠핑장 데이터를 요청해서 가져오기
def search_campgrounds(query="계곡"):
    url = f"http://apis.data.go.kr/B551011/GoCamping/basedList?serviceKey={CAMP_API_KEY}&numOfRows=20&pageNo=1&MobileOS=ETC&MobileApp=camp-app&_type=json&keyword={query}"
    response = requests.get(url)
    if response.status_code != 200:
        return "캠핑장 정보를 가져오지 못했습니다."

    items = response.json().get('response', {}).get('body', {}).get('items', {}).get('item', [])
    
    # 주요 정보만 정리
    result = ""
    for item in items:
        result += f"{item.get('facltNm', '이름없음')} - {item.get('addr1', '주소없음')}\n"
        if item.get('lineIntro'):
            result += f"📌 소개: {item['lineIntro']}\n"
        if item.get('featureNm'):
            result += f"🌲 특징: {item['featureNm']}\n"
        result += "\n"
    return result or "조건에 맞는 캠핑장을 찾을 수 없습니다."

# ✅ 2. GPT에게 분석 요청
def ask_gpt_for_recommendation(user_question, campground_text):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "다음은 사용자가 요청한 캠핑장 검색 결과야. 이 중에서 추천할 만한 곳을 골라주고, 이유와 팁도 함께 설명해줘."},
            {"role": "user", "content": f"❓ 사용자의 질문: {user_question}"},
            {"role": "user", "content": f"🔍 검색된 캠핑장 정보:\n{campground_text}"}
        ],
        temperature=0.6,
        max_tokens=1024
    )
    return response.choices[0].message.content

# ✅ 3. 실행부
if __name__ == "__main__":
    user_question = "계곡이 있는 캠핑장을 두어개 추천해줘"
    print(f"🧾 사용자 질문: {user_question}\n")

    campground_data = search_campgrounds(query="계곡")
    print("🏕️ 캠핑장 검색 결과 (요약):\n", campground_data[:1000], "...(중략)...\n")

    gpt_response = ask_gpt_for_recommendation(user_question, campground_data)
    print("📊 GPT 추천 결과:\n")
    print(gpt_response)


🧾 사용자 질문: 계곡이 있는 캠핑장을 두어개 추천해줘

🏕️ 캠핑장 검색 결과 (요약):
 조건에 맞는 캠핑장을 찾을 수 없습니다. ...(중략)...

📊 GPT 추천 결과:

죄송하지만, 현재 계곡이 있는 캠핑장에 대한 정보가 없네요. 대신, 계곡이 있는 캠핑장을 찾을 때 고려할 수 있는 몇 가지 팁을 드리겠습니다.

1. **리뷰 확인**: 캠핑장을 선택할 때는 다른 캠퍼들의 리뷰를 확인하는 것이 중요합니다. 계곡이 아름답고 물이 깨끗한지, 접근성이 좋은지 등을 체크할 수 있습니다.

2. **위치 검색**: 자연이 풍부한 지역, 예를 들어 강이나 계곡이 많은 산악 지역에서 캠핑장을 검색해 보세요. 대표적인 장소로는 지리산, 설악산, 내장산 등이 있습니다.

3. **캠핑장 시설**: 계곡 근처의 캠핑장일 경우, 물놀이가 가능한 시설이나 샤워실, 화장실 등의 편의시설이 잘 갖춰져 있는지를 확인하세요.

4. **계절 고려**: 여름철에는 계곡에서 물놀이를 즐길 수 있지만, 겨울철에는 얼음이 얼 수 있으니 계절에 따라 적절한 캠핑장을 선택하세요.

이런 팁들을 활용하여 원하는 캠핑장을 찾아보시길 바랍니다!


In [1]:
import os
from openai import OpenAI
from pykrx import stock
import pandas as pd
from datetime import datetime, timedelta

# ✅ OpenAI API 키 직접 지정
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ✅ 1. 삼성전자 주가 데이터 가져오기
def get_samsung_data(days=10):
    end = datetime.today()
    start = end - timedelta(days=days * 2)  # 주말 제외 여유분 확보
    df = stock.get_market_ohlcv_by_date(
        start.strftime("%Y%m%d"), end.strftime("%Y%m%d"), "005930"
    )
    df.reset_index(inplace=True)
    df = df.tail(days)  # 최근 N일 데이터만 선택
    df = df[['날짜', '시가', '고가', '저가', '종가', '거래량']]
    return df

# ✅ 2. 데이터를 텍스트로 정리 (날짜별 요약)
def convert_data_to_text(df):
    result = ""
    for _, row in df.iterrows():
        result += f"{row['날짜'].strftime('%Y-%m-%d')}: 시가 {row['시가']:,}, 고가 {row['고가']:,}, 저가 {row['저가']:,}, 종가 {row['종가']:,}, 거래량 {row['거래량']:,}\n"
    return result

# ✅ 3. GPT에게 분석 요청
def ask_gpt_analysis(data_text):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "다음은 삼성전자의 날짜별 주가 데이터야. 이 데이터를 분석해서 보고해줘. 추세, 특징적인 변동, 이상치, 투자자 관점에서 인사이트도 포함해줘."
            },
            {
                "role": "user",
                "content": data_text
            }
        ],
        temperature=0.5,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

# ✅ 4. 실행부
if __name__ == "__main__":
    df_samsung = get_samsung_data(days=10)
    text_data = convert_data_to_text(df_samsung)
    print("📝 GPT에게 보낸 주가 데이터:\n", text_data[:1000], "...(중략)...\n")

    analysis = ask_gpt_analysis(text_data)
    print("📊 GPT 분석 결과:\n")
    print(analysis)


📝 GPT에게 보낸 주가 데이터:
 2025-07-09: 시가 61,400, 고가 61,400, 저가 60,200, 종가 60,400, 거래량 17,533,415
2025-07-10: 시가 60,600, 고가 61,200, 저가 60,400, 종가 61,000, 거래량 14,768,473
2025-07-11: 시가 61,300, 고가 62,800, 저가 61,200, 종가 62,600, 거래량 19,140,650
2025-07-14: 시가 62,300, 고가 62,600, 저가 61,800, 종가 62,500, 거래량 13,563,846
2025-07-15: 시가 62,300, 고가 63,800, 저가 62,000, 종가 63,700, 거래량 18,442,202
2025-07-16: 시가 63,700, 고가 64,700, 저가 63,100, 종가 64,700, 거래량 23,042,660
2025-07-17: 시가 65,900, 고가 66,800, 저가 64,400, 종가 66,700, 거래량 39,448,683
2025-07-18: 시가 67,000, 고가 67,800, 저가 66,300, 종가 67,100, 거래량 23,951,531
2025-07-21: 시가 67,400, 고가 68,800, 저가 67,200, 종가 67,800, 거래량 17,723,148
2025-07-22: 시가 68,100, 고가 68,500, 저가 67,000, 종가 67,300, 거래량 6,974,601
 ...(중략)...

📊 GPT 분석 결과:

### 삼성전자 주가 데이터 분석 보고서 (2025년 7월 9일 ~ 2025년 7월 22일)

#### 1. **주가 추세 분석**
- **전체 추세**: 주가는 2025년 7월 9일 60,400원에서 시작하여 7월 22일에는 67,300원으로 마감했습니다. 이 기간 동안 주가는 지속적으로 상승하는 추세를 보였으며, 총 6,900원의 상승폭을 기록했습니다.
- **상승세**: 7월 11일부터 7월 22일까지의 데이터에서 주가는 연속적